<a href="https://colab.research.google.com/github/mori01-22/kennkyu/blob/main/Sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# タンス検出 (二値分類) ノートブック

このノートブックは、与えられた画像（タンスあり / タンスなし）を識別するための学習・評価・推論パイプラインを示します。主に TensorFlow / Keras の転移学習（MobileNetV2）を利用します。

前提: 画像データはローカルに用意され、以下のようなフォルダ構成になっていることを想定します:

```
data/
  train/
    tansu/        # タンスが写っている画像（約1000枚程度）
    not_tansu/    # タンスが写っていない画像
```

(検証は自動で分割します)

ノート: このノートブックは Colab / ローカルどちらでも動きます。ローカル環境で GPU を使う場合は適切に TensorFlow をインストールしてください。

In [1]:
# 基本的なインポートと設定
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print('TensorFlow version:', tf.__version__)

# 再現性のためのシード設定（任意）
SEED = 123
tf.random.set_seed(SEED)
np.random.seed(SEED)

TensorFlow version: 2.20.0


In [ ]:
# データセットのパスとパラメータ（必要に応じて変更）
data_dir = Path('data/train')  # ここに tansu と not_tansu のフォルダがある想定
img_size = (224, 224)
batch_size = 32

# ヘルパ: デモ用の簡易画像データを生成（data/demo_train に作成）
def create_demo_data(base_dir, img_size=(224,224), n_per_class=12):
    from PIL import Image, ImageDraw
    base = Path(base_dir)
    (base / 'tansu').mkdir(parents=True, exist_ok=True)
    (base / 'not_tansu').mkdir(parents=True, exist_ok=True)
    w, h = img_size
    for i in range(n_per_class):
        # タンスっぽい四角を描いた画像
        img = Image.new('RGB', (w, h), color=(150, 120, 90))
        d = ImageDraw.Draw(img)
        d.rectangle([int(w*0.2), int(h*0.2), int(w*0.8), int(h*0.9)], fill=(80, 60, 40))
        img.save(base / 'tansu' / f'tansu_{i}.jpg', 'JPEG')
        # タンスなし（丸）画像
        img2 = Image.new('RGB', (w, h), color=(200, 220, 240))
        d2 = ImageDraw.Draw(img2)
        d2.ellipse([int(w*0.3), int(h*0.3), int(w*0.7), int(h*0.7)], fill=(120, 140, 160))
        img2.save(base / 'not_tansu' / f'not_{i}.jpg', 'JPEG')
    print(f'Created demo dataset at: {base.resolve()} (each class: {n_per_class} images)')

# フォルダ存在確認・デモ作成（不足時）
if not data_dir.exists() or not any(p.is_dir() for p in data_dir.iterdir()):
    print('警告: data/train が見つからない、またはクラスフォルダが不足しています。デモ用データを生成します。')
    demo_dir = Path('data/demo_train')
    if not demo_dir.exists() or not any(p.is_dir() for p in demo_dir.iterdir()):
        create_demo_data(demo_dir, img_size=img_size, n_per_class=12)
    data_dir = demo_dir

# クラスフォルダ数の確認（label_mode='binary' の場合は 2 クラス必須）
subdirs = [p.name for p in data_dir.iterdir() if p.is_dir()]
print('Found class folders:', subdirs)
if len(subdirs) < 2:
    raise ValueError(f'二値分類には少なくとも2つのクラスフォルダが必要です。見つかったクラス: {subdirs}')

# データ読み込み (80% training, 20% validation) - TensorFlow のユーティリティを使用
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='training',
    seed=SEED,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='validation',
    seed=SEED,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print('Classes:', class_names)


Found 0 files belonging to 0 classes.


ValueError: When passing `label_mode="binary"`, there must be exactly 2 class_names. Received: class_names=[]

In [ ]:
# パフォーマンス最適化: キャッシュとプリフェッチ
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# サンプル画像を表示して確認
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[int(labels[i])])
        plt.axis('off')
plt.show()

In [ ]:
# データ拡張レイヤー（学習時のみ適用）
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name='data_augmentation')

# 正規化レイヤー (MobileNetV2 では入力を [-1,1] にスケーリングするのが一般的)
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# モデル構築: 転移学習(MobileNetV2) + 軽いヘッド
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet')
base_model.trainable = False  # まずは凍結してヘッドのみ学習

inputs = keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# コールバック: 早期終了とモデル保存
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint('best_tansu_model.h5', save_best_only=True)
]

# 学習（ヘッドのみ）
epochs = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)

In [ ]:
# 評価と学習曲線の表示
import matplotlib.pyplot as plt
acc = history.history.get('accuracy', [])
val_acc = history.history.get('val_accuracy', [])
loss = history.history.get('loss', [])
val_loss = history.history.get('val_loss', [])
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='train_acc')
plt.plot(epochs_range, val_acc, label='val_acc')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='train_loss')
plt.plot(epochs_range, val_loss, label='val_loss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.show()

# 精度の表示
loss_val, acc_val = model.evaluate(val_ds)
print(f'Validation loss: {loss_val:.4f}, accuracy: {acc_val:.4f}')

In [ ]:
# 任意: ベースモデルを微調整（ファインチューニング）する場合の例
# base_model.trainable = True
# fine_tune_at = 100  # 層のどこから微調整するか
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False
# model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
# fine_history = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=callbacks)

# 保存済みモデルをロードして単一画像で推論する例
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model, img_size=img_size):
    img = image.load_img(img_path, target_size=img_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    pred = model.predict(x)[0][0]
    return pred  # 0..1 (sigmoid) -> 閾値 0.5 で分類

# 使い方例（デモ用、適切な画像パスに置き換えてください）
# sample_pred = predict_image('data/sample.jpg', model)
# print('Pred (0..1):', sample_pred)
# print('Pred label:', 'tansu' if sample_pred>=0.5 else 'not_tansu')

# モデル保存（SavedModel 形式）
model.save('tansu_detector_saved_model')
print('モデルを tansu_detector_saved_model に保存しました')

## データ準備の注意点

- クラス不均衡がある場合は、データ拡張やクラス重みの調整を検討してください。
- もし“タンスが写っているか”だけでなく位置も知りたい場合は、物体検出（例: YOLO, Faster R-CNN）を検討してください。
- 画像サイズはモデルに合わせて調整できますが、224×224 は MobileNet 系の一般的な設定です。

## 次の改善案

1. クラス重みの使用や Focal Loss で不均衡対応。
2. K-fold クロスバリデーションで安定した評価。
3. 推論高速化のために変換 (TF Lite) を作成。


In [ ]:
# GPU 確認と（可能なら）混合精度の有効化（GPU を使う場合に実行してください）
from tensorflow.keras import mixed_precision

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print('GPU devices detected:')
    for g in gpus:
        print('  -', g)
    try:
        mixed_precision.set_global_policy('mixed_float16')
        print('Mixed precision policy set to mixed_float16 (faster on supported GPUs).')
    except Exception as e:
        print('Could not enable mixed precision:', e)
else:
    print('No GPU detected. The notebook will run on CPU.')

# 推奨: GPUがある場合はバッチサイズを大きめに設定 (例: 64)
print('\nIf using GPU, consider increasing batch_size in the notebook (e.g., 64) for better throughput.')

In [ ]:
# --- 追加: クラス重み、自動コールバック拡張、評価指標、TF Lite 変換 ---
# このセルは以下を行います:
# 1) data/train 配下のファイル数からクラス重みを計算
# 2) ReduceLROnPlateau, TensorBoard をコールバックに追加
# 3) 検証データセット上で混同行列・分類レポート・ROC を出力
# 4) SavedModel から TF Lite へ変換（最適化を適用）

from collections import Counter
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# 1) クラス数のカウント (data_dir はノートブックで定義済み)
try:
    counts = {}
    for cls in class_names:
        p = Path(data_dir)/cls
        counts[cls] = len(list(p.glob('*'))) if p.exists() else 0
    print('Class counts:', counts)
    total = sum(counts.values())
    n_classes = len(class_names)
    class_weight = {}
    for i, cls in enumerate(class_names):
        cnt = counts[cls]
        class_weight[i] = (total / (n_classes * cnt)) if cnt > 0 else 1.0
    print('Computed class_weight:', class_weight)
except Exception as e:
    print('Could not compute class weights from data_dir:', e)
    class_weight = None

# 2) 追加コールバック
callbacks += [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7),
    keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1)
]
print('Callbacks:', [type(c).__name__ for c in callbacks])

# 3) 検証データで予測して評価（予測は確率と閾値0.5）
print('\nEvaluating on validation set and computing metrics...')
y_true = []
y_prob = []
for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy().astype(int).tolist())
    y_prob.extend(preds.flatten().tolist())

# 二値閾値での予測
y_pred = [1 if p >= 0.5 else 0 for p in y_prob]

print('\nClassification report:')
try:
    print(classification_report(y_true, y_pred, target_names=class_names))
except Exception as e:
    print('Could not compute classification_report:', e)

# 混同行列の表示
try:
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title('Confusion Matrix')
    plt.show()
except Exception as e:
    print('Could not compute or plot confusion matrix:', e)

# ROC-AUC と ROC 曲線
try:
    auc_score = roc_auc_score(y_true, y_prob)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC = {auc_score:.3f}')
    plt.plot([0,1],[0,1],'k--')
    plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Curve'); plt.legend()
    plt.show()
    print('ROC AUC:', auc_score)
except Exception as e:
    print('ROC AUC could not be computed:', e)

# 4) TF Lite 変換 (SavedModel が存在することが前提)
print('\nAttempting TF Lite conversion from SavedModel...')
import tensorflow as _tf
try:
    converter = _tf.lite.TFLiteConverter.from_saved_model('tansu_detector_saved_model')
    converter.optimizations = [_tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    open('tansu_detector.tflite', 'wb').write(tflite_model)
    print('Saved tansu_detector.tflite')
except Exception as e:
    print('TF Lite conversion failed:', e)

# 補足: クラス重みを用いて再学習したい場合は下記を有効にしてください
# model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=callbacks, class_weight=class_weight)
